# Site location mapper for the United Kingdom

### Import statements

In [ ]:
# show maps in separate window
import tkinter as tk
import matplotlib
%matplotlib qt
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import numpy as np
from tkinter.filedialog import asksaveasfilename
import csv

In [16]:
class SiteLocator(tk.Tk):
    
    def __init__(self):
        super().__init__()  # attributes of tk.Tk
        # Window properties
        self.title("Industrial Site Locator")
        self.geometry("1000x800")
        # Map variables
        self.geoweight = tk.IntVar(self, value=100)
        self.popweight = tk.IntVar(self, value=100)
        self.transweight = tk.IntVar(self, value=100)
        # Currently displayed raster variable
        self.current_raster = [0]
        # Figure variable
        self.fig = matplotlib.figure.Figure(figsize=(5, 8))  # do not use plt figure, that pops up second window
        self.ax = self.fig.add_subplot(111, visible=False)  # do not show until first map shows up
        # Frames
        self.infoframe = tk.Frame(self, bg="white")
        self.infoframe.pack(side="left", expand=1, fill="both")
        self.mapframe = tk.Frame(self, bg="white")
        self.mapframe.pack(side="right", expand=1, fill="both")
        # Text labels
        # Title of the application
        # Bold adapted from https://stackoverflow.com/a/46495200/18668457
        self.maintext = tk.Label(self.infoframe, text="Industrial Site\nLocator", bg="white", borderwidth=0, highlightthickness=0, font=("Arial", 30, "bold"), fg="#31a354")
        self.maintext.place(relx=0.1, rely=0.1, relheight=0.15, relwidth=0.8, anchor="nw")
        # Explanatory text
        self.explanation = tk.Label(self.infoframe, text="Use the scrollbars to adjust the importance of factors.\nYou can display and save the maps to .csv files.", bg="white", borderwidth=0, highlightthickness=0, font=("Arial", 10))
        self.explanation.place(relx=0.1, rely=0.25, relheight=0.1, relwidth=0.8, anchor="nw")
        # Scales
        # Highlightthickness adapted from https://stackoverflow.com/a/4311134/18668457
        self.geoscale = tk.Scale(self.infoframe, from_=0, to=100, orient="horizontal", variable=self.geoweight, label="Geology", bg="white", troughcolor="#a1d99b", bd=0, highlightthickness=0, activebackground="white", font=("Arial", 10, 'bold'))
        self.geoscale.place(relx=0.1, rely=0.4, relheight=0.1, relwidth=0.8, anchor="nw")
        self.popscale = tk.Scale(self.infoframe, from_=0, to=100, orient="horizontal", variable=self.popweight, label="Population", bg="white", troughcolor="#a1d99b", bd=0, highlightthickness=0, activebackground="white", font=("Arial", 10, 'bold'))
        self.popscale.place(relx=0.1, rely=0.5, relheight=0.1, relwidth=0.8, anchor="nw")
        self.transscale = tk.Scale(self.infoframe, from_=0, to=100, orient="horizontal", variable=self.transweight, label="Transport", bg="white", troughcolor="#a1d99b", bd=0, highlightthickness=0, activebackground="white", font=("Arial", 10, 'bold'))
        self.transscale.place(relx=0.1, rely=0.6, relheight=0.1, relwidth=0.8, anchor="nw")
        # Display button
        self.displaybutton = tk.Button(self.infoframe, text="Display map", command=self.display, bg="#31a354", bd=1, fg="white")
        self.displaybutton.place(relx=0.1, rely=0.8, relheight=0.1, relwidth=0.35, anchor="nw")
        # Saving functionality
        self.savebutton = tk.Button(self.infoframe, text="Save map", command=self.save, bg="red", bd=1, fg="white")
        self.savebutton.place(relx=0.55, rely=0.8, relheight=0.1, relwidth=0.35, anchor="nw")
        # Canvas
        self.canvas = matplotlib.backends.backend_tkagg.FigureCanvasTkAgg(self.fig, master=self.mapframe)
        self.canvas._tkcanvas.place(relx=0, rely=0, relheight=1, relwidth=1, anchor="nw")

    def display(self):
        rasters = self.raster_reader()
        weights = self.weight_reader()
        avg_raster = self.raster_overlayer(rasters, weights)
        self.current_raster = avg_raster
        self.shower(avg_raster)

    def shower(self, raster: list) -> None:
        self.ax.set_visible(True)  # set subplot visible with the correct axis
        img = self.ax.imshow(raster, cmap="gray")
        self.canvas.draw()

    def raster_reader(self) -> list:
        rasters = [np.genfromtxt("geology.csv", dtype=int, delimiter=","), np.genfromtxt("population.csv", dtype=int, delimiter=","), \
            np.genfromtxt("transport.csv", dtype=int, delimiter=",")]
        return rasters

    def weight_reader(self) -> list:
        weights = [self.geoweight.get(), self.popweight.get(), self.transweight.get()]
        return weights

    def raster_overlayer(self, rasters: list, weights: list) -> list:
        # Zip function from https://stackoverflow.com/a/1663826/18668457
        # np.multiply from https://numpy.org/doc/stable/reference/generated/numpy.multiply.html
        weighted_rasters = [np.multiply(raster, weight) for raster, weight in zip(rasters, weights)]
        # Take average of the weighted rasters
        avg_ras = np.sum(weighted_rasters, axis=0) / len(weighted_rasters)
        # Amax adapted from https://numpy.org/doc/stable/reference/generated/numpy.amax.html
        # Astype adapted from https://appdividend.com/2020/05/06/how-to-convert-numpy-float-to-int-array-in-python/#:~:text=To%20convert%20numpy%20float%20to%20int%20array%20in%20Python%2C%20use,it%20into%20an%20integer%20array.
        rounded_ras = np.round(np.multiply(avg_ras, 255/np.amax(avg_ras))).astype(int)
        return rounded_ras

    def save(self):
        filename = asksaveasfilename(defaultextension=".csv", filetypes=[("Comma Separated Values", "*.csv")])
        with open(filename, "w") as f:
            writer = csv.writer(f)
            for line in self.current_raster:
                writer.writerow(line)
        tk.messagebox.showinfo("Industrial Site Locator - information", "Your map has been saved as {}".format(filename))
        

In [17]:
# Main function to initialise object and start execution
def main():
    root = SiteLocator()  # new SiteLocator object
    root.mainloop()

if __name__ == "__main__":
    main()